In [6]:
import os

import tensorflow as tf
import keras_bert
from functools import partial
import importlib
import data_loader
importlib.reload(data_loader)
print(tf.__version__)
DataLoader = data_loader.DataLoader

config = './config/hub3-1.json'

2.0.0


In [7]:
dl = DataLoader(config, data='../data', n_max=10, test=True)

INFO:root:DataLoader initializing
INFO:root:reading lab data from ../data/soundAttGAN & ../data/soundAttGAN/koreancorpus.xlsx
INFO:root:total number of data: 1925
INFO:root:reading hub data from ../data/KsponSpeech_01
DEBUG:root:../data/KsponSpeech_01/KsponSpeech_0094
INFO:root:total number of data: 709
INFO:root:Reading vocab from ../data/bert_model/vocab.txt
INFO:root:The number of vocab is 119547
INFO:root:Build done
INFO:root:{'batch_size': 32,
 'dropout_rate': 0.5,
 'epochs': 1000,
 'fmax': 8000,
 'hop_length': 250,
 'kernel_size': 4,
 'learning_rate': 0.0001,
 'max_len': 32,
 'max_sec': 2,
 'n_fft': 510,
 'n_max': 100,
 'n_mels': 128,
 'output_size': 256,
 'sr_hub': 16000,
 'sr_lab': 22050,
 'top_db': 80.0,
 'win_length': 510,
 'window': 'hann'}


In [8]:
it = iter(dl.train_generator(data='lab', return_text=True))

In [10]:
dl.data_lab

[{'fileName': '../data/soundAttGAN/20160426111632_2.wav',
  'text': '안녕하세요 아저씨.'},
 {'fileName': '../data/soundAttGAN/20160426111632_4.wav',
  'text': '아저씨 지금, 일 어디 해요?'},
 {'fileName': '../data/soundAttGAN/20160426111632_6.wav',
  'text': '나는 지금 시흥시 정왕동 시국= 시가(시화) 퐁당(공단) 저기 산바이() 선하() 서기 일 하고 있어요.'},
 {'fileName': '../data/soundAttGAN/20160426111632_7.wav',
  'text': '아저씨 자기소개는(자기소개를), 부탁드려요?'},
 {'fileName': '../data/soundAttGAN/20160426111632_9.wav', 'text': '자기소개는.'},
 {'fileName': '../data/soundAttGAN/20160426111632_11.wav', 'text': '인트로덕션?'},
 {'fileName': '../data/soundAttGAN/20160426111632_12.wav',
  'text': '인트로덕션, 어~ 괜찮아요.'},
 {'fileName': '../data/soundAttGAN/20160426111632_14.wav',
  'text': '나는 지(지금), 나라는 네팔입니다.'},
 {'fileName': '../data/soundAttGAN/20160426111632_16.wav',
  'text': '네팔 어디에요? '},
 {'fileName': '../data/soundAttGAN/20160426111632_17.wav',
  'text': '나도 네팔 사람이에요.'},
 {'fileName': '../data/soundAttGAN/20160426111632_19.wav',
  'text': '만나서 반갑습니다!'},
 {'fileNa

In [11]:
"""
발음(철자) -> 철자 사용 : 교정이 목적이기 때문에
    발음() -> 공백인 경우는 발음 사용
모든 punctuation 제거
<웃음/허허>, <> -> 제거
((-)) -> ?

"""

'\n발음(철자) -> 철자 사용 : 교정이 목적이기 때문에\n    발음() -> 공백인 경우는 발음 사용\n모든 punctuation 제거\n<웃음/허허>, <> -> 제거\n((-)) -> ?\n\n'

In [13]:
import pandas as pd

In [15]:
koreancorpus= pd.read_excel('../data/soundAttGAN/koreancorpus.xlsx')

In [19]:
koreancorpus['text'] = koreancorpus['text'].values

In [25]:
corpus = koreancorpus['text'].values

In [26]:
corpus

array(['안녕하세요 아저씨.', '아저씨 지금, 일 어디 해요?',
       '나는 지금 시흥시 정왕동 시국= 시가(시화) 퐁당(공단) 저기 산바이() 선하() 서기 일 하고 있어요.', ...,
       '어~ 중국 구향(고향) 가고 싶어요.', '어~ 지금 과찬아요(괜찮아요).',
       '음~ 우리 지페(집에) 어~ 짜주() 똔장찌개(된장찌개), 진지지개(김치찌개) 혼자 만들어도 돼요.'],
      dtype=object)

In [47]:
import unicodedata
import re

In [42]:
def _is_punctuation(char):
    cp = ord(char)
    if ((cp >= 33 and cp <= 47) or (cp >= 58 and cp <= 64) or
            (cp >= 91 and cp <= 96) or (cp >= 123 and cp <= 126)):
        return True
    cat = unicodedata.category(char)
    if cat.startswith("P"):
        return True
    return False



In [77]:
def _prep_fn(text):
    pattern = '[가-힣]+[(]{1}[가-힣]+[)]{1}'
    regex = re.compile(pattern)
    
    find_list = regex.findall(text)
    for each in find_list:
        sub = each.split('(')
        sub = sub[-1][0:-1]
        text = text.replace(each, sub)
        
    text = re.sub('[<]{1}.+[>]{1}', '', text)
    ret_text = []
    for each in text:
        if _is_punctuation(each):
            continue
        ret_text.append(each)
    ret_text = ''.join(ret_text).split()
    return ' '.join(ret_text)
    
    
    

In [78]:
_prep_fn('어~ 우리는, 어~ 두 분 일이 있어서, 어~ 천구(친구)  adasd   타러(데리러) 오 ((-)), 아 그냥, 그거 줄거리예요. <웃음/허허>')

'어 우리는 어 두 분 일이 있어서 어 친구 adasd 데리러 오 아 그냥 그거 줄거리예요'

In [79]:
_prep_fn('친삼초예요(친삼촌이에요)?')

'친삼촌이에요'

In [85]:
corpus_prep = pd.Series(map(_prep_fn, corpus))

In [86]:
corpus_prep.head()

0                                       안녕하세요 아저씨
1                                  아저씨 지금 일 어디 해요
2    나는 지금 시흥시 정왕동 시국 시화 공단 저기 산바이 선하 서기 일 하고 있어요
3                                 아저씨 자기소개를 부탁드려요
4                                           자기소개는
dtype: object

In [87]:
corpus_prep.tail()

1920                음 한국 지금 한국 음식을 다 봐 봐요
1921                  처음 한국 와요 한국 음식이 맛없어
1922                       어 중국 고향 가고 싶어요
1923                            어 지금 괜찮아요
1924    음 우리 집에 어 짜주 된장찌개 김치찌개 혼자 만들어도 돼요
dtype: object

In [88]:
koreancorpus.head()

,fileName,suffix,personId,text,start,end,sex,langNat,learnKor,levKor
0,20160426111632,2,352,안녕하세요 아저씨.,16.973680,18.323045,M,네팔어,0,중
1,20160426111632,4,352,"아저씨 지금, 일 어디 해요?",19.397028,21.568234,M,네팔어,0,중
2,20160426111632,6,353,나는 지금 시흥시 정왕동 시국= 시가(시화) 퐁당(공단) 저기 산바이() 선하() ...,22.416234,31.678996,M,네팔어,0,중
3,20160426111632,7,352,"아저씨 자기소개는(자기소개를), 부탁드려요?",31.678996,34.952234,M,네팔어,0,중
4,20160426111632,9,352,자기소개는.,35.341556,36.088234,M,네팔어,0,중


In [89]:
koreancorpus['text'] = corpus_prep

In [90]:
koreancorpus.head()

,fileName,suffix,personId,text,start,end,sex,langNat,learnKor,levKor
0,20160426111632,2,352,안녕하세요 아저씨,16.973680,18.323045,M,네팔어,0,중
1,20160426111632,4,352,아저씨 지금 일 어디 해요,19.397028,21.568234,M,네팔어,0,중
2,20160426111632,6,353,나는 지금 시흥시 정왕동 시국 시화 공단 저기 산바이 선하 서기 일 하고 있어요,22.416234,31.678996,M,네팔어,0,중
3,20160426111632,7,352,아저씨 자기소개를 부탁드려요,31.678996,34.952234,M,네팔어,0,중
4,20160426111632,9,352,자기소개는,35.341556,36.088234,M,네팔어,0,중


In [107]:
with pd.io.excel.ExcelWriter(path='../data/soundAttGAN/koreancorpus_prep.xlsx') as writer:
    koreancorpus.to_excel(writer)

In [93]:
!pip install openpyxl

     |████████████████████████████████| 174kB 168kB/s eta 0:00:01
  Created wheel for openpyxl: filename=openpyxl-3.0.0-py2.py3-none-any.whl size=241187 sha256=4ef52f2e42880a5f539059df5f7ed47a3fa22f0e4f38df6b2eba0dd6232ccc96
  Stored in directory: /home/mhlee/.cache/pip/wheels/34/ee/6c/1279f7b70ea72432c2cef15dd3d915477cb3771d1618f6b8ef
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-cp37-none-any.whl size=8917 sha256=818740ac792abf56fbd3d0d467ed3c6b19fa3bc7bc3ae47a7cf6ed77e5bcc44e
  Stored in directory: /home/mhlee/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile


In [108]:
pd.read_excel('../data/soundAttGAN/koreancorpus_prep.xlsx').head()

,Unnamed: 0,fileName,suffix,personId,text,start,end,sex,langNat,learnKor,levKor
0,0,20160426111632,2,352,안녕하세요 아저씨,16.973680,18.323045,M,네팔어,0,중
1,1,20160426111632,4,352,아저씨 지금 일 어디 해요,19.397028,21.568234,M,네팔어,0,중
2,2,20160426111632,6,353,나는 지금 시흥시 정왕동 시국 시화 공단 저기 산바이 선하 서기 일 하고 있어요,22.416234,31.678996,M,네팔어,0,중
3,3,20160426111632,7,352,아저씨 자기소개를 부탁드려요,31.678996,34.952234,M,네팔어,0,중
4,4,20160426111632,9,352,자기소개는,35.341556,36.088234,M,네팔어,0,중


In [96]:
!ls ../data/soundAttGAN/

20160109153903_10.wav	20160118134478_19.wav	20161208176486_139.wav
20160109153903_100.wav	20160118134478_190.wav	20161208176486_14.wav
20160109153903_102.wav	20160118134478_192.wav	20161208176486_141.wav
20160109153903_104.wav	20160118134478_193.wav	20161208176486_143.wav
20160109153903_105.wav	20160118134478_195.wav	20161208176486_145.wav
20160109153903_106.wav	20160118134478_196.wav	20161208176486_146.wav
20160109153903_108.wav	20160118134478_198.wav	20161208176486_148.wav
20160109153903_109.wav	20160118134478_199.wav	20161208176486_150.wav
20160109153903_110.wav	20160118134478_2.wav	20161208176486_152.wav
20160109153903_111.wav	20160118134478_20.wav	20161208176486_154.wav
20160109153903_113.wav	20160118134478_200.wav	20161208176486_156.wav
20160109153903_115.wav	20160118134478_202.wav	20161208176486_158.wav
20160109153903_117.wav	20160118134478_203.wav	20161208176486_16.wav
20160109153903_119.wav	20160118134478_22.wav	20161208176486_160.wav
20160109153903_121.wav	20160118134478_23.w